In [110]:
import numpy as np
import os, tempfile
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from scipy import stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
%matplotlib inline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

In [111]:
pd17 = pd.read_csv("./data/2017nfldata.csv")
pd18 = pd.read_csv("./data/2018nfldata.csv")
pd19 = pd.read_csv("./data/2019nfldata.csv")
pd20 = pd.read_csv("./data/2020nfldata.csv")
pd21 = pd.read_csv("./data/2021nfldata.csv")

In [112]:
pd17.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Todd Gurley*+,LAR,RB,23,15,15,0,0,0,...,NaN,NaN,319.0,383.3,391.3,351.3,192.0,1,1.0,GurlTo01
1,2,Le'Veon Bell*+,PIT,RB,25,15,15,0,0,0,...,NaN,NaN,257.0,341.6,349.6,299.1,130.0,2,2.0,BellLe00
2,3,Kareem Hunt*,KAN,RB,22,16,16,0,0,0,...,NaN,NaN,242.0,295.2,302.2,268.7,115.0,3,3.0,HuntKa00
3,4,Alvin Kamara*,NOR,RB,22,16,3,0,0,0,...,1.0,NaN,239.0,320.4,327.4,279.9,112.0,4,4.0,KamaAl00
4,5,Melvin Gordon,LAC,RB,24,16,16,0,0,0,...,NaN,NaN,230.0,288.1,294.1,259.1,103.0,5,5.0,GordMe00


In [113]:
pd18.head(3)

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Todd Gurley*+,LAR,RB,24,14,14,0,0,0,...,3.0,NaN,313.0,372.1,379.1,342.6,178.0,1,1.0,GurlTo01
1,2,Saquon Barkley*,NYG,RB,21,16,16,0,0,0,...,1.0,NaN,295.0,385.8,391.8,340.3,159.0,2,2.0,BarkSa00
2,3,Christian McCaffrey,CAR,RB,22,16,16,1,1,50,...,NaN,NaN,279.0,385.5,392.5,332.0,143.0,3,3.0,McCaCh01


In [114]:
pd19.head(3)

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Christian McCaffrey*+,CAR,RB,23,16,16,0,2,0,...,1.0,NaN,355.0,471.2,477.2,413.2,215.0,1,1.0,McCaCh01
1,2,Lamar Jackson*+,BAL,QB,22,15,15,265,401,3127,...,NaN,NaN,416.0,415.7,429.7,421.7,152.0,1,2.0,JackLa00
2,3,Derrick Henry *,TEN,RB,25,15,15,0,0,0,...,NaN,NaN,277.0,294.6,303.6,285.6,136.0,2,3.0,HenrDe00


In [115]:
pd20.head(3)

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Derrick Henry *+,TEN,RB,26,16,16,0,0,0,...,1.0,NaN,314.0,333.1,341.1,323.6,184.0,1,1.0,HenrDe00
1,2,Alvin Kamara*,NOR,RB,25,15,10,0,0,0,...,NaN,NaN,295.0,377.8,383.8,336.3,165.0,2,2.0,KamaAl00
2,3,Dalvin Cook*,MIN,RB,25,14,14,0,0,0,...,3.0,NaN,294.0,337.8,346.8,315.8,164.0,3,3.0,CookDa01


In [116]:
pd21.head(3)

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Jonathan Taylor*+,IND,RB,22,17,17,0.0,0.0,0.0,...,NaN,NaN,333.0,373.1,381.1,353.1,187.0,1,1.0,TaylJo02
1,2,Cooper Kupp*+,LAR,WR,28,17,17,0.0,1.0,0.0,...,1.0,NaN,295.0,439.5,442.5,367.0,173.0,1,2.0,KuppCo00
2,3,Deebo Samuel*+,SFO,WR,25,16,15,1.0,2.0,24.0,...,NaN,NaN,262.0,339.0,347.0,300.5,140.0,2,3.0,SamuDe00


In [117]:
for col in pd17.columns:
    print(col)

Rk
Player
Tm
FantPos
Age
G
GS
Cmp
Att
Yds
TD
Int
Att.1
Yds.1
Y/A
TD.1
Tgt
Rec
Yds.2
Y/R
TD.2
Fmb
FL
TD.3
2PM
2PP
FantPt
PPR
DKPt
FDPt
VBD
PosRank
OvRank
-9999


In [118]:
for col in pd18.columns:
    print(col)

Rk
Player
Tm
FantPos
Age
G
GS
Cmp
Att
Yds
TD
Int
Att.1
Yds.1
Y/A
TD.1
Tgt
Rec
Yds.2
Y/R
TD.2
Fmb
FL
TD.3
2PM
2PP
FantPt
PPR
DKPt
FDPt
VBD
PosRank
OvRank
-9999


In [119]:
pd17 = pd17.drop(columns=['Rk', 'Tm', 'Player', 'FantPos', '-9999', 'FantPt', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])
pd18 = pd18.drop(columns=['Rk', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])
pd19 = pd19.drop(columns=['Rk', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])
pd20 = pd20.drop(columns=['Rk', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])
pd21 = pd21.drop(columns=['Rk', 'FantPt', 'PPR', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])

In [120]:
pd17.fillna(0, inplace=True)
pd18.fillna(0, inplace=True)
pd19.fillna(0, inplace=True)
pd20.fillna(0, inplace=True)
pd21.fillna(0, inplace=True)


In [121]:
pd17.head()

,Age,G,GS,Cmp,Att,Yds,TD,Int,Att.1,Yds.1,...,Rec,Yds.2,Y/R,TD.2,Fmb,FL,TD.3,2PM,2PP,PPR
0,23,15,15,0,0,0,0,0,279,1305,...,64,788,12.31,6,5.0,2,19,0.0,0.0,383.3
1,25,15,15,0,0,0,0,0,321,1291,...,85,655,7.71,2,3.0,2,11,0.0,0.0,341.6
2,22,16,16,0,0,0,0,0,272,1327,...,53,455,8.58,3,1.0,1,11,0.0,0.0,295.2
3,22,16,3,0,0,0,0,0,120,728,...,81,826,10.20,5,1.0,1,14,1.0,0.0,320.4
4,24,16,16,0,0,0,0,0,284,1105,...,58,476,8.21,4,1.0,0,12,0.0,0.0,288.1


## Dummy Model

In [122]:
##for dummy model insert prediction simply based on last years rankings? ie Jonathon Taylor #1, Cooper Kupp #2 etc

## Dense Neural Network Model

In [123]:
X = pd17.drop(columns=['PPR'])#Actually need to be 2018 data?
y = pd17['PPR']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [124]:
dense_model = models.Sequential()
dense_model.add(layers.Dense(128, activation='relu', input_dim=23))
dense_model.add(layers.Dense(64, activation='relu'))
dense_model.add(layers.Dense(1, activation='sigmoid'))

dense_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               3072      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [125]:
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [126]:
 model.fit(X_train, y_train, epochs=100, batch_size=128)

Epoch 1/100
4/4 [==============================] - 0s 3ms/step - loss: -136.8712 - accuracy: 0.1046
Epoch 2/100
4/4 [==============================] - 0s 997us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 3/100
4/4 [==============================] - 0s 499us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 4/100
4/4 [==============================] - 0s 748us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 5/100
4/4 [==============================] - 0s 499us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 6/100
4/4 [==============================] - 0s 498us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 7/100
4/4 [==============================] - 0s 376us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 8/100
4/4 [==============================] - 0s 499us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 9/100
4/4 [==============================] - 0s 499us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 10/100
4/4 [==============================] - 0s 499us/step - loss: -136.8712 

4/4 [==============================] - 0s 126us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 81/100
4/4 [==============================] - 0s 498us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 82/100
4/4 [==============================] - 0s 498us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 83/100
4/4 [==============================] - 0s 673us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 84/100
4/4 [==============================] - 0s 0s/step - loss: -136.8712 - accuracy: 0.1046
Epoch 85/100
4/4 [==============================] - 0s 502us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 86/100
4/4 [==============================] - 0s 498us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 87/100
4/4 [==============================] - 0s 499us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 88/100
4/4 [==============================] - 0s 403us/step - loss: -136.8712 - accuracy: 0.1046
Epoch 89/100
4/4 [==============================] - 0s 499us/step - loss: -136.8712 - acc